In [1]:
!pip install bs4 selenium -q

In [2]:
!pip install -q "great-expectations==0.18.19"

In [17]:
import pandas as pd
import great_expectations as ge
from great_expectations.core import ExpectationConfiguration
from great_expectations.dataset import PandasDataset

In [29]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# 1 transaction_fact

In [38]:
df = pd.read_csv("transaction_fact.csv")
df

,transID,payCardID,halteID_tapIn,halteID_tapOut,corridorID,transaction_created_at,tapInTime,tapOutTime,duration,stopStartSeq,stopEndSeq,seqAmount,payAmount
0,EIIW227B8L34VB,180062659848800,P00142,P00253,5,2023-04-03,2023-04-03 05:21:44,2023-04-03 06:00:53,0 days 00:39:09,7,12.0,5.0,3500.0
1,LGXO740D2N47GZ,4885331907664776,B01963P,B03307P,6C,2023-04-03,2023-04-03 05:42:44,2023-04-03 06:40:01,0 days 00:57:17,13,21.0,8.0,3500.0
2,DJWR385V2U57TO,4996225095064169,B00499P,B04962P,R1A,2023-04-03,2023-04-03 05:59:06,2023-04-03 06:50:55,0 days 00:51:49,38,39.0,1.0,3500.0
3,JTUZ800U7C86EH,639099174703,B05587P,B03090P,11D,2023-04-03,2023-04-03 05:44:51,2023-04-03 06:28:16,0 days 00:43:25,23,29.0,6.0,3500.0
4,VMLO535V7F95NJ,570928206772,P00239,P00098,12,2023-04-03,2023-04-03 06:17:35,2023-04-03 06:57:03,0 days 00:39:28,5,15.0,10.0,3500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37895,ZWEC949B8Q87QG,4685818286724028395,P00261,P00228,6B,2023-04-21,2023-04-21 18:18:37,2023-04-21 19:55:49,0 days 01:37:12,2,13.0,11.0,3500.0
37896,YHHK837P6Y95GN,6502902290603767,P00064,P00179,9N,2023-04-18,2023-04-18 21:52:31,2023-04-18 22:28:22,0 days 00:35:51,1,2.0,1.0,3500.0
37897,YXPP627N4G95HO,213159426675861,B02873P,B00226P,1T,2023-04-04,2023-04-04 10:29:47,2023-04-04 13:27:25,0 days 02:57:38,12,14.0,2.0,20000.0
37898,RGVK175U2U98UV,377840859133591,B02505P,B01787P,JAK.13,2023-04-15,2023-04-15 19:59:26,2023-04-15 20:27:50,0 days 00:28:24,33,34.0,1.0,0.0


In [41]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'transcation4'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'coba2'
path_to_data = 'transaction_fact.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [42]:
# Creat an expectation suite
expectation_suite_name = 'expectation-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
df = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
df.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,transID,payCardID,halteID_tapIn,halteID_tapOut,corridorID,transaction_created_at,tapInTime,tapOutTime,duration,stopStartSeq,stopEndSeq,seqAmount,payAmount
0,EIIW227B8L34VB,180062659848800,P00142,P00253,5,2023-04-03,2023-04-03 05:21:44,2023-04-03 06:00:53,0 days 00:39:09,7,12.0,5.0,3500.0
1,LGXO740D2N47GZ,4885331907664776,B01963P,B03307P,6C,2023-04-03,2023-04-03 05:42:44,2023-04-03 06:40:01,0 days 00:57:17,13,21.0,8.0,3500.0
2,DJWR385V2U57TO,4996225095064169,B00499P,B04962P,R1A,2023-04-03,2023-04-03 05:59:06,2023-04-03 06:50:55,0 days 00:51:49,38,39.0,1.0,3500.0
3,JTUZ800U7C86EH,639099174703,B05587P,B03090P,11D,2023-04-03,2023-04-03 05:44:51,2023-04-03 06:28:16,0 days 00:43:25,23,29.0,6.0,3500.0
4,VMLO535V7F95NJ,570928206772,P00239,P00098,12,2023-04-03,2023-04-03 06:17:35,2023-04-03 06:57:03,0 days 00:39:28,5,15.0,10.0,3500.0


In [44]:
# 1.memerika data unik
df.expect_column_values_to_be_unique("transID")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 37900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# 2 user_dimension

In [45]:
df1 = pd.read_csv("user_dimension.csv")
df1

,paycard_id,paycardbank,paycardname,paycardsex,paycardbirthdate
0,180062659848800,emoney,Bajragin Usada,M,2008
1,4885331907664776,dki,Gandi Widodo,F,1997
2,4996225095064169,dki,Emong Wastuti,F,1992
3,639099174703,flazz,Surya Wacana,F,1978
4,570928206772,flazz,Embuh Mardhiyah,M,1982
...,...,...,...,...,...
1995,4685818286724028395,brizzi,Kamila Mahendra,F,2004
1996,6502902290603767,dki,Titi Siregar,M,1974
1997,213159426675861,emoney,drg. Zahra Nashiruddin,F,1976
1998,377840859133591,emoney,Ana Agustina,M,1976


In [46]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'user'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'dimension'
path_to_data = 'user_dimension.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [47]:
# Creat an expectation suite
expectation_suite_name = 'expectation-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
df1 = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
df1.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,paycard_id,paycardbank,paycardname,paycardsex,paycardbirthdate
0,180062659848800,emoney,Bajragin Usada,M,2008
1,4885331907664776,dki,Gandi Widodo,F,1997
2,4996225095064169,dki,Emong Wastuti,F,1992
3,639099174703,flazz,Surya Wacana,F,1978
4,570928206772,flazz,Embuh Mardhiyah,M,1982


In [48]:
# 1.memerika data unik
df1.expect_column_values_to_be_unique("paycard_id")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# 3 corridor_dimension

In [53]:
df3 = pd.read_csv("corridor_dimension.csv")
df3

,corridorID,corridorName
0,5,Matraman Baru - Ancol
1,6C,Stasiun Tebet - Karet via Patra Kuningan
2,R1A,Pantai Maju - Kota
3,11D,Pulo Gebang - Pulo Gadung 2 via PIK
4,12,Tanjung Priok - Pluit
...,...,...
520,unknown,Tanah Abang - Meruya
521,2Q,unknown
522,unknown,Pinang Ranti - Pluit
523,unknown,Rusun Cipinang Besar Selatan - Penas Kalimalang


In [54]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'corridor1'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'cordim1'
path_to_data = 'corridor_dimension.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [57]:
# Creat an expectation suite
expectation_suite_name = 'expectation-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
df3 = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
df3.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,corridorID,corridorName
0,5,Matraman Baru - Ancol
1,6C,Stasiun Tebet - Karet via Patra Kuningan
2,R1A,Pantai Maju - Kota
3,11D,Pulo Gebang - Pulo Gadung 2 via PIK
4,12,Tanjung Priok - Pluit


In [58]:
# 1.memerika data unik
df3.expect_column_values_to_be_unique("corridorID")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "element_count": 525,
    "unexpected_count": 507,
    "unexpected_percent": 96.57142857142857,
    "partial_unexpected_list": [
      "5",
      "6C",
      "R1A",
      "11D",
      "12",
      "1T",
      "unknown",
      "B14",
      "1Q",
      "5M",
      "M7B",
      "9A",
      "4",
      "JAK.46",
      "S22",
      "13",
      "9E",
      "M9",
      "JAK.88",
      "T11"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 96.57142857142857,
    "unexpected_percent_nonmissing": 96.57142857142857
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# 4 halte_dimension

In [59]:
df4 = pd.read_csv("halte_dimension.csv")
df4

,halte_ID,halte_Name,halte_City,halteLat,halteLon
0,P00142,Pal Putih,Jakarta Pusat,-6.184631,106.84402
1,B01963P,Kemenkes 2,Jakarta Selatan,-6.228700,106.83302
2,B00499P,Gg. Kunir II,Jakarta Barat,-6.133132,106.81435
3,B05587P,Taman Elok 1,Jakarta Timur,-6.195743,106.93526
4,P00239,Sunter Boulevard Barat,Jakarta Utara,-6.149650,106.88900
...,...,...,...,...,...
3596,B04740P,Simpang Bundaran Ikan Harapan Indah Raya 1,Luar Jakarta,-6.181458,106.98134
3597,B03342P,SDN Cengkareng Barat 05,Jakarta Barat,-6.149878,106.72239
3598,P00284,Stasiun Jatinegara Arah Selatan,Jakarta Timur,-6.215541,106.87436
3599,B02471P,MNC Studios,Jakarta Barat,-6.190376,106.76625


In [60]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'halte'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'haldim'
path_to_data = 'halte_dimension.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [61]:
# Creat an expectation suite
expectation_suite_name = 'expectation-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
df4 = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
df4.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,halte_ID,halte_Name,halte_City,halteLat,halteLon
0,P00142,Pal Putih,Jakarta Pusat,-6.184631,106.84402
1,B01963P,Kemenkes 2,Jakarta Selatan,-6.228700,106.83302
2,B00499P,Gg. Kunir II,Jakarta Barat,-6.133132,106.81435
3,B05587P,Taman Elok 1,Jakarta Timur,-6.195743,106.93526
4,P00239,Sunter Boulevard Barat,Jakarta Utara,-6.149650,106.88900


In [62]:
# 1.memerika data unik
df4.expect_column_values_to_be_unique("halte_ID")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3601,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}